# Сравнение хранилищ ClickHouse и Vertica

In [381]:
import uuid
import random 
import timeit
import time

### Подготовка данных для теста

In [382]:
file = open('insert_file.csv','w')

In [383]:
n = 1
while n <= 10000000:
    file.write(f'{n},{uuid.uuid1()},{uuid.uuid1()},{random.randint(1,20000)}\n')
    n+=1

# Clickhouse

In [384]:
from clickhouse_driver import Client

client = Client(host='127.0.0.1') 

### Создание базы test_base

In [385]:
client.execute('CREATE DATABASE IF NOT EXISTS test_base ON CLUSTER company_cluster')

[('clickhouse-node5', 9000, 0, '', 5, 0),
 ('clickhouse-node1', 9000, 0, '', 4, 0),
 ('clickhouse-node3', 9000, 0, '', 3, 0),
 ('clickhouse-node4', 9000, 0, '', 2, 0),
 ('clickhouse-node2', 9000, 0, '', 1, 0),
 ('clickhouse-node6', 9000, 0, '', 0, 0)]

### Создание таблицы test_table

In [386]:
client.execute('CREATE TABLE IF NOT EXISTS test_base.test_table ON CLUSTER company_cluster (id Int64, movie_id UUID, user_id UUID, frame Int64) Engine=MergeTree() ORDER BY id')

[('clickhouse-node5', 9000, 0, '', 5, 0),
 ('clickhouse-node1', 9000, 0, '', 4, 0),
 ('clickhouse-node3', 9000, 0, '', 3, 0),
 ('clickhouse-node6', 9000, 0, '', 2, 0),
 ('clickhouse-node4', 9000, 0, '', 1, 0),
 ('clickhouse-node2', 9000, 0, '', 0, 0)]

### Вставка из файла

In [387]:
file = open('insert_file.csv','r').read()

In [388]:
start_time = time.time()
client.execute(f"INSERT INTO test_base.test_table (id, movie_id, user_id, frame) FORMAT CSV {file}")
clickhouse_insert = f"Clickhouse вставка из файла: {(time.time() - start_time)}"
print(clickhouse_insert)

Clickhouse вставка из файла: 43.40561008453369


### Вставка нескольких строк

In [389]:
start_time = time.time()
client.execute(f'''INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES
               (1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096),
               (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798),
               (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)''')
clickhouse_insert3 = f"Clickhouse вставка 3 строк: {(time.time() - start_time)}"
print(clickhouse_insert3)

Clickhouse вставка 3 строк: 0.010514974594116211


### Многократная вставка нескольких строк с использованием timeit

In [390]:
# код для timeit
timeit_code = """client.execute("INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES (1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096), (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798), (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)")"""

In [391]:
elapsed_time = timeit.timeit(timeit_code, number=1000,globals=globals())/1000

clickhouse_insert3_iterate = f"Clickhouse cреднее время вставки 3 строк 1000 раз: {elapsed_time})"
print(clickhouse_insert3_iterate)

Clickhouse cреднее время вставки 3 строк 1000 раз: 0.011105239316999358)


### Вставка 500 строк

In [392]:
insert_list = [[i,uuid.uuid1(),uuid.uuid1(),random.randint(1,20000)] for i in range(500)]

In [393]:
start_time = time.time()
client.execute(f"INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES", 
                (tuple(row) for row in insert_list))
clickhouse_insert500 = f"Clickhouse время вставки 500 строк: {(time.time() - start_time)}"
print(clickhouse_insert500)

Clickhouse время вставки 500 строк: 0.015022039413452148


### Многократная вставка 500 строк с использованием timeit

In [394]:
timeit_code = """client.execute(f"INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES", 
                (tuple(row) for row in insert_list))"""

In [395]:
elapsed_time = timeit.timeit(timeit_code, number=1000,globals=globals())/1000
clickhouse_insert500_iterate = f"Clickhouse cреднее время вставки 500 строк 1000 раз: {elapsed_time}"
print(clickhouse_insert500_iterate)

Clickhouse cреднее время вставки 500 строк 1000 раз: 0.011780224458001613


## Запросы

### Запрос всех данных

In [ ]:
start_time = time.time()
result = client.execute('SELECT * FROM test_base.test_table')
res_time = time.time() - start_time
print(f"Clickhouse время запроса всех строк: {res_time}")

In [ ]:
clickhouse_select_all = f"Clickhouse время запроса всех строк: {res_time} kоличество строк в ответе: {len(result)}"
print(clickhouse_select_all)
result.clear()

### Запрос по movie_id

In [ ]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE movie_id =='e67bfbe0-0b3f-11ed-98f2-acde48001122'")
res_time = time.time() - start_time
clickhouse_select_movie_id = f"Clickhouse время запроса по конкретному movie_id: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_movie_id)
result.clear()

### Запрос с фильтацией по frame

In [ ]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE frame > 10000")
res_time = time.time() - start_time
clickhouse_select_frame1 = f"Clickhouse время запроса frame > 1000: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_frame1)
result.clear()

### Запрос с фильтацией по frame с меньшим количеством строк в ответе

In [ ]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE frame > 100 and frame < 300")
res_time = time.time() - start_time
clickhouse_select_frame100_300 = f"Clickhouse время запроса frame > 100 and < 300: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_frame100_300)
result.clear()

### Предыдущий запрос c order by

In [ ]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE frame > 100 and frame <300 order by frame")
res_time = time.time() - start_time
clickhouse_select_frame100_300_order = f"Clickhouse время запроса frame > 100 and < 300 order: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_frame100_300_order)
result.clear()

### Удаление таблицы test_base.test_table

In [ ]:
client.execute('DROP TABLE IF EXISTS test_base.test_table ON CLUSTER company_cluster')

# Vertica

In [ ]:
import vertica_python
connection_info = {
    'host': '127.0.0.1',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}

In [ ]:
connection = vertica_python.connect(**connection_info)
cursor = connection.cursor()

In [ ]:
cursor.execute("""CREATE TABLE test (
        id IDENTITY,
        id_test INTEGER,
        movie_id UUID,
        user_id UUID,
        frame INTEGER NOT NULL);
    """)

### Вставка из файла

In [ ]:
start_time = time.time()
cursor.execute("COPY test FROM LOCAL 'insert_file.csv' PARSER fcsvparser();")
vertica_insert = f"Vertica вставка из файла: {(time.time() - start_time)}"
print(vertica_insert)

### Вставка нескольких строк

In [ ]:
start_time = time.time()
cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                  [(1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096),
                   (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798),
                   (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)],
                    use_prepared_statements=True)
vertica_insert3 = f"Vertica вставка 3 строк: {(time.time() - start_time)}"
print(vertica_insert3)

### Многократная вставка нескольких строк с использованием timeit

In [ ]:
# код для timeit
timeit_code = """cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                  [(1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096),
                   (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798),
                   (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)],
                    use_prepared_statements=True)"""

In [ ]:
elapsed_time = timeit.timeit(timeit_code, number=1000,globals=globals())/1000

vertica_insert3_iterate = f"Vertica cреднее время вставки 3 строк 1000 раз: {elapsed_time})"
print(vertica_insert3_iterate)

### Вставка 500 строк

In [ ]:
insert_list = [(i,uuid.uuid1(),uuid.uuid1(),random.randint(1,20000)) for i in range(500)]

In [ ]:
start_time = time.time()
cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                insert_list,
                use_prepared_statements=True)

vertica_insert500 = f"Vertica время вставки 500 строк: {(time.time() - start_time)}"
print(vertica_insert500)

### Многократная вставка 500 строк с использованием timeit

In [ ]:
# код для timeit
timeit_code = """cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                insert_list,
                use_prepared_statements=True)"""

In [ ]:
elapsed_time = timeit.timeit(timeit_code, number=10,globals=globals())/10

vertica_insert500_iterate = f"Vertica cреднее время вставки 500 строк 10 раз: {elapsed_time})"
print(vertica_insert500_iterate)
# время выполнение блока при параметре number=1000 примерно 4000 секунд а это больше часа... 

## Запросы

### Запрос всех данных

In [ ]:
start_time = time.time()
result = cursor.execute('SELECT * FROM test').fetchall()
res_time = time.time() - start_time
print(f"Vertica время запроса всех строк: {res_time}")

In [ ]:
vertica_select_all = f"Vertica время запроса всех строк: {res_time} kоличество строк в ответе: {len(result)}"
print(vertica_select_all)
result.clear()

### Запрос по movie_id

In [ ]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE movie_id ='e67bfbe0-0b3f-11ed-98f2-acde48001122'")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_movie_id = f"Vertica время запроса по конкретному movie_id: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_movie_id)
result.clear()

### Запрос с фильтацией по frame

In [ ]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE frame > 10000")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_frame1 = f"Vertica время запроса по конкретному movie_id: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_frame1)
result.clear()

### Запрос с фильтацией по frame с меньшим количеством строк в ответе

In [ ]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE frame > 100 and frame < 300")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_frame100_300 = f"Vertica время запроса frame > 100 and < 300: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_frame100_300)
result.clear()

### Предыдущий запрос c order by

In [ ]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE frame > 100 and frame < 300 order by frame")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_frame100_300_order = f"Vertica время запроса frame > 100 and < 300 order: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_frame100_300_order)
result.clear()

In [ ]:
cursor.execute("DROP TABLE test;")

# Итог

In [ ]:
print(clickhouse_insert,
      "",
clickhouse_insert3,
      "",
clickhouse_insert3_iterate,
      "",
clickhouse_insert500,
      "",
clickhouse_insert500_iterate,
      "",
clickhouse_select_all,
      "",
clickhouse_select_movie_id,
      "",
clickhouse_select_frame1,
      "",
clickhouse_select_frame100_300,
      "",
clickhouse_select_frame100_300_order,
      "",
vertica_insert,
      "",
vertica_insert3,
      "",
vertica_insert3_iterate,
      "",
vertica_insert500,
      "",
vertica_insert500_iterate,
      "",
vertica_select_all,
      "",
vertica_select_movie_id,
      "",
vertica_select_frame1,
      "",
vertica_select_frame100_300,
      "",
vertica_select_frame100_300_order,
      "",
      sep='\n')

### По результатам clickhouse работает быстрее vertica, так же из результатов видно что в clickhouse время вставки 3 строк занимает практически такое же время как и вставка 500 строк поэтому данные стоит вставлять пачками.

In [423]:
print(clickhouse_insert,
      "",
clickhouse_insert3,
      "",
clickhouse_insert3_iterate,
      "",
clickhouse_insert500,
      "",
clickhouse_insert500_iterate,
      "",
clickhouse_select_all,
      "",
clickhouse_select_movie_id,
      "",
clickhouse_select_frame1,
      "",
clickhouse_select_frame100_300,
      "",
clickhouse_select_frame100_300_order,
      "",
vertica_insert,
      "",
vertica_insert3,
      "",
vertica_insert3_iterate,
      "",
vertica_insert500,
      "",
vertica_insert500_iterate,
      "",
vertica_select_all,
      "",
vertica_select_movie_id,
      "",
vertica_select_frame1,
      "",
vertica_select_frame100_300,
      "",
vertica_select_frame100_300_order,
      "",
      sep='\n')

Clickhouse вставка из файла: 43.40561008453369

Clickhouse вставка 3 строк: 0.010514974594116211

Clickhouse cреднее время вставки 3 строк 1000 раз: 0.011105239316999358)

Clickhouse время вставки 500 строк: 0.015022039413452148

Clickhouse cреднее время вставки 500 строк 1000 раз: 0.011780224458001613

Clickhouse время запроса всех строк: <module 'time' (built-in)> kоличество строк в ответе: 10503433

Clickhouse время запроса по конкретному movie_id: 0.08047294616699219 количество строк в ответе: 1001

Clickhouse время запроса frame > 1000: 27.43072509765625 количество строк в ответе: 5264882

Clickhouse время запроса frame > 100 and < 300: 0.5426690578460693 количество строк в ответе: 105387

Clickhouse время запроса frame > 100 and < 300 order: 0.7501430511474609 количество строк в ответе: 105387

Vertica вставка из файла: 38.90188384056091

Vertica вставка 3 строк: 0.14099693298339844

Vertica cреднее время вставки 3 строк 1000 раз: 0.04308059314799902)

Vertica время вставки 500 с

### По результатам clickhouse работает быстрее vertica, так же из результатов видно что в clickhouse время вставки 3 строк занимает практически такое же время как и вставка 500 строк поэтому данные стоит вставлять пачками.